In [43]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from tqdm import tqdm

In [44]:
#Path
quant_table_path = 'C:/Users/huberr/Desktop/Data_Luis/RH110_MeOH_P1_P2_0723_quant.csv'#here put the path to the folder where your quant table is.
metadata_table_path ='C:/Users/huberr/Desktop/Data_Luis/RH110_P1_P2_MeOH_metadata_full.csv'#here put the path to the folder where your quant table is.
final_path ='C:/Users/huberr/Desktop/Data_Luis/Output/RH110/'#here put the path to the folder where your want to save your tables and htmls
experiment = 'RH110_0723_final' #put the unique code to recognize this set of data


### 1) Process the quantitative data

In [45]:
#recover correlation groups 
correlation_groups_df = pd.read_csv(quant_table_path, sep=',')#,  index_col='row ID')
correlation_groups_df.rename(columns = lambda x: x.replace(' Peak area', ''),inplace=True) #Lambda pour renommer toutes les colonnes, cherche "Peak area" dans ttes les colonnes et remplace par ""
correlation_groups_df.drop(list(correlation_groups_df.filter(regex = 'Unnamed: ')), axis = 1, inplace = True) #axis 1=column, axis 0=row. Efface ttes les colonnes appelées "unnamed"
correlation_groups_df.drop(['row ion mobility', ''
    'row ion mobility unit', 'row CCS', 
    'correlation group ID', 'auto MS2 verify',
    'identified by n=', 'partners'], axis=1, inplace=True)
correlation_groups_df.rename(columns={'best ion': 'adduct (ion identity)', 'neutral M mass':'neutral mass (ion identity)', 'row retention time':'retention time (min)' }, inplace=True)
correlation_groups_df['retention time (min)'] = correlation_groups_df['retention time (min)'].round(decimals=2)
correlation_groups_df['row m/z'] = correlation_groups_df['row m/z'].round(decimals=4)

#complete correlation groups
correlation_groups_df['annotation network number'] = correlation_groups_df['annotation network number'].fillna(correlation_groups_df['row ID'].apply(str) + 'x')

quant_df = correlation_groups_df.copy()
quant_df.drop(['row ID', 'row m/z', 'retention time (min)', 'adduct (ion identity)','neutral mass (ion identity)'], axis=1, inplace=True) #df avec uniquement annot net number et intensités

#aggregate metadata columns according to correlations groups
correlation_groups_df = correlation_groups_df.iloc[:, :6] #colonnes avec row ID, row m/z, rt, adduct, neutral mass, annotation network number
agg_func = {'retention time (min)': 'mean', 'row m/z': set,  'adduct (ion identity)': set, 'row ID': set, 'neutral mass (ion identity)': 'max'} #voir notes 
dfcg = correlation_groups_df.groupby('annotation network number', as_index=False).agg(agg_func)
dfcg[['adduct (ion identity)', 'row ID']] = dfcg[['adduct (ion identity)', 'row ID']].astype(str)  

# reduced quant_df and groupby annotation network number
dfq = quant_df.groupby('annotation network number', as_index=False).max() #garde la valeur du ion d'un groupe IIN le plus intense

#add the metadata of the annotation groups
dff = pd.merge(dfq, dfcg[['annotation network number', 'retention time (min)', 'row m/z', 'row ID', 'adduct (ion identity)','neutral mass (ion identity)']], how ='left', left_on = 'annotation network number', right_on='annotation network number')
dff.fillna({'adduct (ion identity)': 'not available', 'neutral mass (ion identity)': 'not available'}, inplace=True)
dff.head(2) #montre les deux premières lignes du tableau
#dff.shape #indique les dimensions du tableau

,annotation network number,20220922_LFX_RH_005_RH110_P1_SPEMeOH_A05_1_pos.mzML,20220922_LFX_RH_009_RH110_P1_SPEMeOH_A09_1_pos.mzML,20220922_LFX_RH_006_RH110_P1_SPEMeOH_A06_1_pos.mzML,20220922_LFX_RH_002_RH110_P1_SPEMeOH_A01_1_pos.mzML,20220922_LFX_RH_004_RH110_P1_SPEMeOH_A04_1_pos.mzML,20220922_LFX_RH_011_RH110_P1_SPEMeOH_A11_1_pos.mzML,20220922_LFX_RH_018_RH110_P1_SPEMeOH_B06_1_pos.mzML,20220922_LFX_RH_012_RH110_P1_SPEMeOH_A12_1_pos.mzML,20220922_LFX_RH_008_RH110_P1_SPEMeOH_A08_1_pos.mzML,...,20220922_LFX_RH_094_RH110_P2_SPEMeOH_H10_1_pos.mzML,20220922_LFX_RH_096_RH110_P2_SPEMeOH_H12_1_pos.mzML,20220922_LFX_RH_085_RH110_P2_SPEMeOH_H01_1_pos.mzML,20220922_LFX_RH_063_RH110_P2_SPEMeOH_F03_1_pos.mzML,20220922_LFX_RH_090_RH110_P2_SPEMeOH_H06_1_pos.mzML,retention time (min),row m/z,row ID,adduct (ion identity),neutral mass (ion identity)
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,13847.882,0.0,0.00,0.0,1.153333,"{257.186, 279.1683, 535.3468}","{168, 171, 176}","{'[M+Na]+', '[M+H]+', '[2M+Na]+'}",256.17887
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,366411.6,0.0,0.0,...,152483.98,158869.060,0.0,354900.28,418596.0,1.225000,"{321.1478, 243.1341}","{206, 199}","{'[M+H]+', '[M+DMSO+H]+'}",242.1267


In [46]:
#read the metadata table
metadata_df = pd.read_csv(metadata_table_path, sep=';')
metadata_df.head(2)

,filename,Type,Strain,Rep,full_name,species
0,20220922_LFX_RH_019_RH110_P2_SPEMeOH_B07_1_pos...,blank,634,rep1,blank_634_rep1,Sclerotinia sp.(634)
1,20220922_LFX_RH_051_RH110_P2_SPEMeOH_E03_1_pos...,blank,634,rep2,blank_634_rep2,Sclerotinia sp.(634)


In [47]:
#merge the metadata with the quant table grouped by 'annotation network numbers'
dft=dfq.copy()
dft.set_index('annotation network number', inplace=True)
dft=dft.transpose()
dft.reset_index(inplace=True)
dft.rename(columns={'index': 'filename'}, inplace=True)
dft=pd.merge(dft, metadata_df, how ='left', left_on ='filename', right_on='filename')
dft.to_csv(final_path+experiment+'_merged_full.csv')
dft

,filename,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,...,994x,998x,999x,99x,9x,Type,Strain,Rep,full_name,species
0,20220922_LFX_RH_005_RH110_P1_SPEMeOH_A05_1_pos...,0.000,0.00,627586.25,0.00,0.000,0.00,0.00,0.0,0.00,...,0.00,0.000,0.00,0.00,21001.256,blank,1263,rep1,blank_1263_rep1,B. pseudocinerea(1263)
1,20220922_LFX_RH_009_RH110_P1_SPEMeOH_A09_1_pos...,0.000,0.00,284096.70,0.00,0.000,0.00,0.00,0.0,0.00,...,0.00,0.000,731427.94,0.00,0.000,blank,1239,rep2,blank_1239_rep2,B. pseudocinerea(1239)
2,20220922_LFX_RH_006_RH110_P1_SPEMeOH_A06_1_pos...,0.000,0.00,673329.20,0.00,0.000,0.00,0.00,0.0,0.00,...,0.00,37075.504,698538.94,0.00,0.000,blank,1239,rep1,blank_1239_rep1,B. pseudocinerea(1239)
3,20220922_LFX_RH_002_RH110_P1_SPEMeOH_A01_1_pos...,0.000,0.00,190574.06,0.00,0.000,0.00,0.00,0.0,0.00,...,0.00,0.000,442753.50,0.00,13277.246,incub,730,rep1,incub_730_rep1,B. pseudocinerea aff.(730)
4,20220922_LFX_RH_004_RH110_P1_SPEMeOH_A04_1_pos...,0.000,0.00,0.00,0.00,0.000,0.00,0.00,0.0,0.00,...,0.00,17054.209,584841.90,0.00,175247.420,incub,2255,rep1,incub_2255_rep1,B. eucalypti cf.(2255)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,20220922_LFX_RH_094_RH110_P2_SPEMeOH_H10_1_pos...,0.000,152483.98,0.00,0.00,0.000,0.00,0.00,0.0,0.00,...,0.00,0.000,379183.62,0.00,786015.300,std,NaN,NaN,std__,NaN
188,20220922_LFX_RH_096_RH110_P2_SPEMeOH_H12_1_pos...,13847.882,158869.06,0.00,0.00,0.000,0.00,0.00,0.0,0.00,...,0.00,0.000,0.00,0.00,496004.620,std,NaN,NaN,std__,NaN
189,20220922_LFX_RH_085_RH110_P2_SPEMeOH_H01_1_pos...,0.000,0.00,684720.06,0.00,94241.740,0.00,0.00,0.0,301235.84,...,0.00,0.000,0.00,0.00,273562.470,incub,1233,rep3,incub_1233_rep3,B. pseudocinerea(1233)
190,20220922_LFX_RH_063_RH110_P2_SPEMeOH_F03_1_pos...,0.000,354900.28,3152977.50,468516.12,75367.086,653464.06,134029.34,5891374.5,618835.20,...,559090.00,0.000,0.00,477234.03,4057623.800,incub,1630,rep3,incub_1630_rep3,B. pseudocinerea(1630)


In [48]:
#erase all the blanks and QC's - Change the string as needed
def drop_samples_based_on_string(df,list_of_strings_for_QC_Blank_filter,column):
    print(df.shape)
    for string in list_of_strings_for_QC_Blank_filter:
        df = df[~df[column].str.contains(string, na=False)]
        df = df.dropna(how = 'any', subset=[column])
    print(df.shape)
    return df

list_of_strings_for_QC_Blank_filter = ['NaN', 'QC'] #this is the string you used to identify the samples, NaN will erase all the samples without metadata after merging.
column_to_use_for_filtering = 'Type' #this information should be included in the metadata table
dft = drop_samples_based_on_string(dft,list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)

(192, 1863)
(192, 1863)


In [49]:
#add a column with the same identifier for the X replicated (it'll be used to separate the table into blanks and incubations)
#dft['name']=dft['Type']+'_'+dft['Strain']

#separate dataframe into blanks and incubations and obtain the mean by replicate
## blanks
dfb= dft[dft.Type == 'blank']
dfb = dfb.groupby('species', as_index=False).mean()
dfb.sort_values(by=['species'], ascending=True)
dfb.set_index('species', inplace=True)
#save file for future plot
dfbf=dfb.copy()
dfbf.reset_index(inplace=True)
#dfbf.rename(columns={'name': 'Strain'}, inplace=True)
dfbf.to_csv(final_path+experiment+'_blanks_average.csv')

#replaced '0' by low vlaue
dfb = dfb.astype(float)
dfb=dfb.mask(dfb<=0, 0.001)

##incubations
dfi= dft[dft.Type == 'incub']
dfi = dfi.groupby('species', as_index=False).mean()
dfi.sort_values(by=['species'], ascending=True)
dfi.set_index('species', inplace=True)
dfi = dfi.astype(float)


#save file for future plot
dfif=dfi.copy()
dfif.reset_index(inplace=True)
#dfif.rename(columns={'name': 'Strain'}, inplace=True)
dfif.to_csv(final_path+experiment+'_incub_average.csv')

#calculate the ratio incubtation/blanks
dfr=dfi/dfb.values
dfr.to_csv(final_path+experiment+'_ratio.csv')
dfr.head(2)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,98x,990x,991x,992x,993x,994x,998x,999x,99x,9x
species,,,,,,,,,,,,,,,,,,,,,
B. aclada(1228),0.0,1.001179,0.46378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.900953,0.861947
B. cinerea(1020),0.0,1.632256,1.17003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.239526


In [50]:
#create the table for fingerprinting based on the ratio conditions 

# change values lower than 20 by '0'
df_plot=dfr.copy().astype(float)
df_plot =df_plot.applymap(lambda x: np.nan if x >= 20 else 0) #remplit le tableau par des 0 pour les ratio < 20 et de NaN pour les >20
#df1.head(2)
df_plot.update(dfi,overwrite=False) #ovwerwrite=false évite de remplacer les 0 que l'on souhaite garder
df_plot.reset_index(inplace=True)
#df_plot.rename(columns={'name': 'Strain'}, inplace=True)
#df1.head(2)
#check dataframe shape
#df1.shape

#drop columns containing only zeros and save the final file
zero_cols = [ col for col, is_zero in ((df_plot == 0).sum() == df_plot.shape[0]).items() if is_zero ]
df_plot.drop(zero_cols, axis=1, inplace=True)

#species_df = pd.read_csv(species_table_path, sep=';')
#df_plot=pd.merge(df_plot, species_df, how ='left', left_on ='Strain', right_on='Strain')
#df_plot.set_index('Strain')
df_plot.sort_values(by=['species'], ascending=True, inplace=True)
df_plot.to_csv(final_path+experiment+'_incub_df_to_plot.csv')
print(df_plot.shape)
df_plot.head()

(27, 1414)


,species,0.0,1.0,3.0,4.0,7.0,8.0,9.0,10.0,11.0,...,978x,97x,987x,989x,98x,993x,994x,998x,999x,99x
0,B. aclada(1228),0.000000,0.000,0.000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00
1,B. cinerea(1020),0.000000,0.000,0.000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00
2,B. cinerea(1260),0.000000,251889.496,9550.259,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,271960.566667,0.0,54917.84
3,B. cinerea(1732),39468.125667,0.000,0.000,1.365462e+06,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00
4,B. cinerea(740),0.000000,0.000,0.000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00


### 2) create the fingerprints

In [51]:
#Plot the treated data for the incubations
df_plot =pd.read_csv(final_path+experiment+'_incub_df_to_plot.csv')
df_plot.drop(['Unnamed: 0'], axis=1, inplace=True)
#create and write fig
fig = px.bar(df_plot, y="species", x=df_plot.columns[1:], 
                orientation="h", template="simple_white", 
                width=800, height=1000, color_discrete_sequence=px.colors.qualitative.Light24)
fig.write_html(final_path+experiment+'_Fingerprint_incub_treated.html')
#fig.write_image(final_path+'Fingerprint_incub_treated.png', engine="kaleido")
#fig.show()

c:\Users\huberr\Anaconda3\lib\site-packages\plotly\express\_core.py:1222: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



In [52]:
#Plot the treated data for the incubations filtered witouth starting compound 

df_plot =pd.read_csv(final_path+experiment+'_incub_df_to_plot.csv')
df_plot.drop(['Unnamed: 0'], axis=1, inplace=True)
df_plot.drop(['109.0'], axis=1, inplace=True) #check the right number of the column
#create and write fig
fig = px.bar(df_plot, y="species", x=df_plot.columns[1:], 
                orientation="h", template="simple_white", 
                width=800, height=1000, color_discrete_sequence=px.colors.qualitative.Light24)
fig.write_html(final_path+experiment+'_Fingerprint_incub_treated_filteredSM.html')
#fig.write_image(final_path+'Fingerprint_incub_treated.png', engine="kaleido")
#fig.show()

In [53]:
#Plot the RAW data for the incubations
df_incub =pd.read_csv(final_path+experiment+'_incub_average.csv')
df_incub.drop(['Unnamed: 0'], axis=1, inplace=True)

#create and write fig
fig = px.bar(df_incub, y="species", x=df_incub.columns[1:], orientation="h", template="simple_white", width=800, height=1000, color_discrete_sequence=px.colors.qualitative.Light24)
fig.write_html(final_path+experiment+'_Fingerprint_incub_raw.html')
#fig.write_image(final_path+'Fingerprint_incub_treated.png', engine="kaleido")
#fig.show()

In [54]:
#Plot the RAW data for the Blanks
df_blank=pd.read_csv(final_path+experiment+'_blanks_average.csv')
df_blank.drop(['Unnamed: 0'], axis=1, inplace=True)

#create and write fig
fig = px.bar(df_blank, y="species", x=df_blank.columns[1:], orientation="h", template="simple_white", width=800, height=1000, color_discrete_sequence=px.colors.qualitative.Light24)
fig.write_html(final_path+experiment+'_Fingerprint_blank_raw.html')
#fig.write_image(final_path+'Fingerprint_incub_treated.png', engine="kaleido")
fig.show()